# WIP Will Be Fixed once Soiling Alg is updated

## Data for soiling seperation test produced using this notebook

### Code from [TAC2_scratchbook.ipynb](https://github.com/slacgismo/solar-data-tools/blob/master/notebooks/TAC2_scratchwork.ipynb) in example notebooks folder

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# PVInsight Code Imports
from solardatatools import DataHandler
from solardatatools.dataio import get_pvdaq_data
from solardatatools.algorithms.soiling import soiling_seperation

In [5]:
data_frame = get_pvdaq_data(sysid=1199, year=[2015, 2016, 2017], api_key='DEMO_KEY')[0]

CAUTION: Multiple scan rates detected!=======================] 100.0% ...queries complete in 22.8 seconds       
Scan rates (in seconds): [300, 280]
0 transitions detected.
Suggest splitting data set between:




In [6]:
data_frame.loc['2017-06-01':] = data_frame.loc['2017-06-01':] * 0.75

In [7]:
dh = DataHandler(data_frame)

In [8]:
dh.run_pipeline(power_col='ac_power')

total time: 11.15 seconds
--------------------------------
Breakdown
--------------------------------
Preprocessing              2.44s
Cleaning                   4.18s
Filtering/Summarizing      4.54s
    Data quality           0.19s
    Clear day detect       0.14s
    Clipping detect        3.50s
    Capacity change detect 0.71s



In [9]:
bix = dh.daily_flags.clear

In [10]:
dh.fit_statistical_clear_sky_model()

----------------------
SCSF Problem Setup
----------------------
Matrix Size: 288 x 1096 = 315648 power measurements
Sparsity: 91.57%
26616 non-zero measurements under clear conditions
Model size: 288 x 6 + 6 x 1096 = 8304 parameters


----------------------
Algorithm Iterations
----------------------
Starting at Objective: 9.338e+09, f1: 1.512e+07, f2: 1.814e+02, f3: 7.521e+09, f4: 1.802e+09
    Minimizing left matrix


/Users/derins/opt/anaconda3/envs/pvi-user/lib/python3.7/site-packages/cvxpy/reductions/solvers/solving_chain.py:167: UserWarning: You are solving a parameterized problem that is not DPP. Because the problem is not DPP, subsequent solves will not be faster than the first one. For more information, see the documentation on Discplined Parametrized Programming, at
	https://www.cvxpy.org/tutorial/advanced/index.html#disciplined-parametrized-programming
  warnings.warn(dpp_error_msg)


    Minimizing right matrix
1 - Objective: 1.708e+07, f1: 1.541e+07, f2: 5.944e+02, f3: 1.668e+06, f4: 4.797e-04
Caution: residuals increased
    Minimizing left matrix
    Minimizing right matrix
2 - Objective: 1.525e+07, f1: 1.354e+07, f2: 1.937e+03, f3: 1.710e+06, f4: 9.238e-06
    Minimizing left matrix
    Minimizing right matrix
3 - Objective: 1.508e+07, f1: 1.338e+07, f2: 1.742e+03, f3: 1.698e+06, f4: 1.533e-06
    Minimizing left matrix
    Minimizing right matrix
4 - Objective: 1.506e+07, f1: 1.337e+07, f2: 1.720e+03, f3: 1.688e+06, f4: 1.633e-06
    Minimizing left matrix
    Minimizing right matrix
5 - Objective: 1.505e+07, f1: 1.336e+07, f2: 1.709e+03, f3: 1.680e+06, f4: 2.068e-05
Minimization complete in 3.15 minutes


In [11]:
pi = np.sum(dh.filled_data_matrix, axis=0) / np.sum(dh.scsf.clear_sky_signals(), axis=0)

### Saves the input variables to csv

In [12]:
np.savetxt('./bix_soiling_seperation_input.csv', bix, delimiter=',')
np.savetxt('./pi_soiling_seperation_input.csv', pi, delimiter=',')

### Saves the outputs tp csv

In [13]:
s1, s2, s3 = soiling_seperation(pi, index_set=bix, iterations=1)

In [14]:
np.savetxt('./s1_soiling_seperation_output.csv', s1, delimiter=',')
np.savetxt('./s2_soiling_seperation_output.csv', s2, delimiter=',')
np.savetxt('./s3_soiling_seperation_output.csv', s3, delimiter=',')